# 基于长词黑名单去除高频广告

In [1]:
import numpy as np

In [2]:
#相邻4个长词，视为异常
thr = 4
history_words = [""] * (thr - 1)
black_list = set()
cnt = 0
with open('vocabs/vocab_5690450.txt') as f:
    for pair in f:
        if pair[0] != '\t' and len(pair.split('\t')[0])==7:
            word = pair.split('\t')[0]
            if not "" in history_words:
                black_list |= set(history_words)
            history_words[cnt]=word
        else:
            history_words[cnt] = ""
        cnt = (cnt + 1) % (thr - 1)

In [3]:
len(black_list)

56039

In [4]:
#随机查看广告词
np.array(list(black_list))[np.random.randint(0,len(black_list),30)]

array(['的申请人和办理', '资金管理第二节', '掉色就是假货了', '苗穴位介入疗法', '血通络康复疗法', '计划及政策措施',
       '八年的时间岁月', '该群员踢出群聊', '官形成与原发癌', '同时全球市场在', '炎研究治疗中心', '料是多种多样的',
       '宁夏银川专业染', '酒店位于基督城', '希望可以有人组', '和查处冒充专利', '影响女性工作和', '策实施医疗减免',
       '各教学指导委员', '一个一个试用着', '及选股涨停技术', '然的传统中药材', '我又和客服说了', '荐的自然之谜海',
       '秋水多年来辛勤', '解他们的症状呢', '表面有黑色粉末', '我来到青岛癫痫', '量增加时可促使', '亚公寓酒店享受'],
      dtype='<U7')

In [5]:
import ahocorasick as ah
aca= ah.Automaton()
for word in black_list:
    #构造自动机时对词频进行过滤
    aca.add_word(word.encode(),None)
aca.make_automaton()

In [6]:
def isad(text):
    cnt = 0
    for end_index,_ in aca.iter(text.encode()):
        cnt += 1
        if cnt >= 3:
            return True
    return False

In [7]:
isad("和预先付费政策   甲诊疗领域始终   室内外保温工程")

True

In [8]:
isad("今天天气真好")

False

In [9]:
with open('high_ff_unique.txt','r') as f_in:
    with open('high_ff_final.txt','w') as f_out:
        for line in f_in:
            if not isad(line):
                print(line,end="",file=f_out)